#**Answer Generation 任務**










### pytorch環境設定(https://pytorch.org)
選擇適合的版本並複製指令貼到本專案

In [1]:
!pip install torch torchvision

In [2]:
#確定有安裝成功
import torch
print(torch.__version__)
torch.cuda.is_available()

1.5.0+cu101


True

### **取得 google drive 存取權限**

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
cd /content/drive/My Drive/Colab Notebooks/Bert/AnswerGeneration/

/content/drive/My Drive/Colab Notebooks/Bert/AnswerGeneration


In [5]:
!ls

DRCD_training.json  train.ipynb


In [0]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

### **下載dataset**

In [7]:
import os
def fileExists(filepath):
  return  os.path.exists(filepath)
#我把資料集放到我的github
if fileExists('train.json')!=True:
  !wget https://raw.githubusercontent.com/harry83528/NLP-BERT-Experience/master/AnswerGeneration/DataSet/DRCD_training.json

--2020-05-30 16:38:34--  https://raw.githubusercontent.com/harry83528/NLP-BERT-Experience/master/AnswerGeneration/DataSet/DRCD_training.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15094666 (14M) [text/plain]
Saving to: ‘DRCD_training.json.1’

DRCD_training.json. 100%[===================>]  14.39M  63.9MB/s    in 0.2s    

2020-05-30 16:38:35 (63.9 MB/s) - ‘DRCD_training.json.1’ saved [15094666/15094666]



In [8]:
!ls

DRCD_training.json  DRCD_training.json.1  train.ipynb


In [9]:
!pip install ijson

In [10]:
import ijson

data = ijson.parse(open('DRCD_training.json', 'r'))
count=0
for prefix, event, value in data:
  print('prefix',prefix)
  print('event',event)
  print('value',value)
  print('---'*50)
  count=count+1
  if count==51:
    break;
        

prefix 
event start_map
value None
------------------------------------------------------------------------------------------------------------------------------------------------------
prefix 
event map_key
value version
------------------------------------------------------------------------------------------------------------------------------------------------------
prefix version
event string
value 1.3
------------------------------------------------------------------------------------------------------------------------------------------------------
prefix 
event map_key
value data
------------------------------------------------------------------------------------------------------------------------------------------------------
prefix data
event start_array
value None
------------------------------------------------------------------------------------------------------------------------------------------------------
prefix data.item
event start_map
value None
------------------

### **安裝所需的函式庫-HuggingFace 團隊將 GitHub 專案大翻新並更名成 transformers(Install the Hugging Face Library)**
https://github.com/huggingface/transformers

In [11]:
!pip install transformers

In [0]:
%tensorflow_version 2.x

### **指定想要用的模型是哪一種**
到以下網址查詢模型與函數資訊
https://huggingface.co/transformers

## **引入需要⽤到的函式庫**

In [0]:
import os
import pickle
import torch
from transformers import BertConfig, BertTokenizer, BertForMaskedLM, AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F # 激勵函數

### **載入資料並將資料轉換成輸入格式(Token Embeddings、Segment Embeddings、Position Embeddings)**

### **輸入格式轉Dataset**

## **確定 Embedding 是否正確**

## **分割數據**
把資料切成訓練集跟測試集
對於 Supervised Learning 來說，我們使⽤標註資料（Labeled Data）來做訓練
因此我們需要訓練資料（Training Data）
但是我們蒐集到的資料卻不能全部拿來做訓練
因為我們必須要保留⼀些當作測試資料（Testing Data）來評估模型表現
這些資料必須跟訓練資料是完全不同的，否則就有作弊的嫌疑
所以我們會把我們的資料切成訓練集與測試集，通常會保留比較多當作訓練資料
（60%~80%），其餘做測試資料

In [0]:
def split_dataset(full_dataset, split_rate=0.8):  
    train_size = int(split_rate * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
    return train_dataset,test_dataset

### **Fine-Tuning**


In [0]:
# 計算正確值
def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

### **開始訓練**

### **儲存模型**